In [1]:
# Import dependencies
import json
from sqlalchemy import create_engine
import pandas as pd
import pymysql
pymysql.install_as_MySQLdb()
from datetime import datetime, date

In [2]:
# Import configuration variables
from config import sf_username, sf_password, sf_security_token
from config import remote_db_endpoint, remote_db_port
from config import remote_db_name, remote_db_user, remote_db_pwd

In [3]:
# Set up simple_salesforce
from simple_salesforce import Salesforce
sf = Salesforce(username=sf_username, password=sf_password, security_token=sf_security_token)

In [4]:
# Connect to your MySQL database
engine = create_engine(f"mysql://{remote_db_user}:{remote_db_pwd}@{remote_db_endpoint}:{remote_db_port}/{remote_db_name}")
conn = engine.connect()

## Query 'classparticipant' table from MySql

In [5]:
# Query Data
class_par = pd.read_sql("SELECT * FROM classparticipant", conn)
class_par.head()

,ID_ClassParticipant,ID_Student,ID_Class,StartDate,EndDate
0,1,33,1,2020-03-16,None
1,2,34,1,2020-03-16,None
2,3,35,1,2020-03-16,None
3,4,62,1,2020-03-16,None
4,5,36,1,2020-03-16,None


## Query Student data from Salesforce in order to Get its 'Record Id'

In [9]:
sf_student_data = sf.query_all_iter("SELECT Id, Name, ID_Student__c FROM Student__c")

In [10]:
student_sf = []

for data in sf_student_data:
    records = {
        'Student_Record_ID': data['Id'],
        'First Name' : data['Name'],
        'ID_Student__c': data['ID_Student__c']
    }
    
    student_sf.append(records)

In [11]:
student_sf

[{'Student_Record_ID': 'a053h000005EPxmAAG',
  'First Name': 'Dino',
  'ID_Student__c': 55.0},
 {'Student_Record_ID': 'a053h000005EPwoAAG',
  'First Name': 'Sarah',
  'ID_Student__c': 43.0},
 {'Student_Record_ID': 'a053h000005EPw0AAG',
  'First Name': 'Heather',
  'ID_Student__c': 33.0},
 {'Student_Record_ID': 'a053h000005EPwtAAG',
  'First Name': 'Undraa',
  'ID_Student__c': 44.0},
 {'Student_Record_ID': 'a053h000005EPxSAAW',
  'First Name': 'Chris',
  'ID_Student__c': 51.0},
 {'Student_Record_ID': 'a053h000005EPy1AAG',
  'First Name': 'Patrick',
  'ID_Student__c': 58.0},
 {'Student_Record_ID': 'a053h000005EPwPAAW',
  'First Name': 'Pariya',
  'ID_Student__c': 38.0},
 {'Student_Record_ID': 'a053h000005EPwjAAG',
  'First Name': 'Jawaher',
  'ID_Student__c': 42.0},
 {'Student_Record_ID': 'a053h000005EPy6AAG',
  'First Name': 'Robert',
  'ID_Student__c': 59.0},
 {'Student_Record_ID': 'a053h000005EPwKAAW',
  'First Name': 'Katherine',
  'ID_Student__c': 37.0},
 {'Student_Record_ID': 'a053

In [12]:
student_sf = pd.DataFrame(student_sf)
student_sf.head()

,Student_Record_ID,First Name,ID_Student__c
0,a053h000005EPxmAAG,Dino,55.0
1,a053h000005EPwoAAG,Sarah,43.0
2,a053h000005EPw0AAG,Heather,33.0
3,a053h000005EPwtAAG,Undraa,44.0
4,a053h000005EPxSAAW,Chris,51.0


In [18]:
#print(student_sf.dtypes)

In [20]:
student_sf['ID_Student__c'] = student_sf['ID_Student__c'].astype(int)

## Prep. 'classparticipant' table to be prompt to join with Class table

In [21]:
# Transform Data to perfectly match between dataframe headers and field names in Salesforce
class_par = class_par.rename(columns={
                                        'ID_ClassParticipant':'ID_ClassParticipant__c',
                                        'ID_Student':'ID_Student__c',
                                        'ID_Class':'ID_Class__c',
                                        'StartDate':'StartDate__c',
                                        'EndDate':'EndDate__c'
                                })
class_par.head()

,ID_ClassParticipant__c,ID_Student__c,ID_Class__c,StartDate__c,EndDate__c
0,1,33,1,2020-03-16,None
1,2,34,1,2020-03-16,None
2,3,35,1,2020-03-16,None
3,4,62,1,2020-03-16,None
4,5,36,1,2020-03-16,None


In [22]:
class_par['StartDate__c'] = class_par['StartDate__c'].apply(lambda x: x.strftime('%Y-%m-%d'))
class_par['EndDate__c'] = class_par['EndDate__c'].apply(lambda x: x.strftime('%Y-%m-%d')) 

AttributeError: 'NoneType' object has no attribute 'strftime'

## Join tables

In [23]:
classpar_x_student = class_par.merge(student_sf)
classpar_x_student.head()

,ID_ClassParticipant__c,ID_Student__c,ID_Class__c,StartDate__c,EndDate__c,Student_Record_ID,First Name
0,1,33,1,2020-03-16,None,a053h000005EPw0AAG,Heather
1,2,34,1,2020-03-16,None,a053h000005EPw5AAG,Ilana
2,3,35,1,2020-03-16,None,a053h000005EPwAAAW,Jessica
3,4,62,1,2020-03-16,None,a053h000005EPyGAAW,Kevin
4,5,36,1,2020-03-16,None,a053h000005EPwFAAW,Kandra


In [24]:
# Convert DataFrame to a list of dictionaries --> This can help facilitate a bulk insert
class_par_load = classpar_x_student.to_dict(orient='record')
class_par_load

[{'ID_ClassParticipant__c': 1,
  'ID_Student__c': 33,
  'ID_Class__c': 1,
  'StartDate__c': '2020-03-16',
  'EndDate__c': None,
  'Student_Record_ID': 'a053h000005EPw0AAG',
  'First Name': 'Heather'},
 {'ID_ClassParticipant__c': 2,
  'ID_Student__c': 34,
  'ID_Class__c': 1,
  'StartDate__c': '2020-03-16',
  'EndDate__c': None,
  'Student_Record_ID': 'a053h000005EPw5AAG',
  'First Name': 'Ilana'},
 {'ID_ClassParticipant__c': 3,
  'ID_Student__c': 35,
  'ID_Class__c': 1,
  'StartDate__c': '2020-03-16',
  'EndDate__c': None,
  'Student_Record_ID': 'a053h000005EPwAAAW',
  'First Name': 'Jessica'},
 {'ID_ClassParticipant__c': 4,
  'ID_Student__c': 62,
  'ID_Class__c': 1,
  'StartDate__c': '2020-03-16',
  'EndDate__c': None,
  'Student_Record_ID': 'a053h000005EPyGAAW',
  'First Name': 'Kevin'},
 {'ID_ClassParticipant__c': 5,
  'ID_Student__c': 36,
  'ID_Class__c': 1,
  'StartDate__c': '2020-03-16',
  'EndDate__c': None,
  'Student_Record_ID': 'a053h000005EPwFAAW',
  'First Name': 'Kandra'},


## Insert 'classparticipant' records

In [25]:
for row in class_par_load:
    
    record = {
        'ID_ClassParticipant__c': row['ID_ClassParticipant__c'],
        'ID_Student__c': row['ID_Student__c'],
        'ID_Class__c':row['ID_Class__c'],
        'StartDate__c':row['StartDate__c'],
        'EndDate__c':row['EndDate__c']
    }
    try:
        sf.Class_Participant__c.create(record)
    except Exception as e:
        print(e)

In [ ]:
'''
# Inserting rows in bulk for Class Participant table on Salesforce
try:
    sf.bulk.Class_Participant__c.insert(class_par_push)
except Exception as e:
    print(e)
'''